In [3]:
# ui/app.py
import streamlit as st
import pandas as pd

ModuleNotFoundError: No module named 'streamlit'

In [ ]:
FINAL_PATH = "final_ranked_candidates.csv"
PAPERS_PATH = "phase3/outputs/phase3_papers.csv"

In [ ]:
st.set_page_config(
    page_title="AI Alzheimer's Drug Discovery",
    layout="wide"
)

---------------------------<br>
Load data<br>
---------------------------

In [ ]:
@st.cache_data
def load_data():
    final_df = pd.read_csv(FINAL_PATH)
    papers_df = pd.read_csv(PAPERS_PATH)
    return final_df, papers_df

In [ ]:
final_df, papers_df = load_data()

---------------------------<br>
Header<br>
---------------------------

In [ ]:
st.title(" AI-Driven Alzheimer's Drug Discovery")
st.markdown("""
We combine **machine learning**, **biological reasoning**, and **automated literature analysis**
to identify promising drug candidates for Alzheimer's disease.
""")

---------------------------<br>
Pipeline explanation<br>
---------------------------

In [ ]:
with st.expander(" How the system works"):
    st.markdown("""
**Phase 1 “ Blood-Brain Barrier Screening**  
Predicts which drugs can cross the BBB using molecular features.

In [ ]:
**Phase 2 “ Mechanism Plausibility**  
Checks whether drugs target Alzheimer-relevant biological mechanisms.

In [ ]:
**Phase 3 “ Literature Intelligence**  
Automatically reads thousands of research papers and scores positive vs negative evidence.

In [ ]:
**Final Output**  
A ranked shortlist of drug candidates with supporting evidence.
""")

In [ ]:
st.divider()

---------------------------<br>
Final ranking table<br>
---------------------------

In [ ]:
st.subheader(" Final Ranked Drug Candidates")

In [ ]:
top_n = st.slider("Number of drugs to show", 5, 50, 15)

In [ ]:
display_cols = [
    "drug_name",
    "final_score",
    "phase2_score",
    "signed_score",
    "net_positive",
    "n_papers",
    "models",
    "confidence"
]

In [ ]:
st.dataframe(
    final_df[display_cols]
    .head(top_n)
    .style.format({
        "final_score": "{:.3f}",
        "phase2_score": "{:.3f}",
        "signed_score": "{:.2f}",
        "confidence": "{:.2f}"
    }),
    use_container_width=True
)

In [ ]:
st.divider()

---------------------------<br>
Drug detail explorer<br>
---------------------------

In [ ]:
st.subheader(" Explore Drug Evidence")

In [ ]:
search_query = st.text_input(" Search for a drug (type to filter)")

In [ ]:
if search_query:
    filtered = final_df[
        final_df["drug_name"]
        .str.contains(search_query, case=False, na=False)
    ]
else:
    filtered = final_df

In [ ]:
selected_drug = st.selectbox(
    "Select a drug",
    filtered["drug_name"].unique()
)

In [ ]:
drug_row = final_df[final_df["drug_name"] == selected_drug].iloc[0]

In [ ]:
col1, col2, col3 = st.columns(3)

In [ ]:
with col1:
    st.metric("Final Score", f"{drug_row['final_score']:.3f}")
with col2:
    st.metric("Phase 2 (Mechanism)", f"{drug_row['phase2_score']:.3f}")
with col3:
    st.metric("Phase 3 (Literature)", f"{drug_row['signed_score']:.2f}")

st.markdown(f
<br>
**Models observed:** `{drug_row['models']}`  <br>
**Positive vs Negative evidence:** `{int(drug_row['net_positive'])}`  <br>


---------------------------<br>
Paper evidence<br>
---------------------------

In [ ]:
st.subheader(" Supporting Literature")

In [ ]:
drug_papers = papers_df[papers_df["drug"] == selected_drug]

In [ ]:
if drug_papers.empty:
    st.info(
        "No Alzheimer-specific preclinical literature was found for this drug. "
        "This may indicate a novel candidate or lack of published studies."
    )